testing Torch backend:
* circuit : oscillator, transmon, fluxonium

In [1]:
from DiSuQ.Torch import models
from torch import tensor
from numpy import arange
from DiSuQ.utils import plotCompare

### LC oscillator

In [2]:
El = 1000
Ec = 5
circuit = models.oscillatorLC([50],El,Ec,sparse=False)

In [3]:
H_LC = circuit.oscillatorHamiltonianLC()
oscillator = circuit.circuitEnergy(H_LC=H_LC)

In [4]:
H_LC = circuit.chargeHamiltonianLC()
charge = circuit.circuitEnergy(H_LC=H_LC)

In [ ]:
oscillator = oscillator.detach().numpy()
charge = charge.detach().numpy()
plotCompare(arange(len(oscillator)),{'oscillator':oscillator,'charge':charge},'Energy Spectrum','levels','E(GHz)')

### Transmon

In [ ]:
Ej,Ec = 20,.3

In [ ]:
basis = {'O':[],'I':[],'J':[100]}
circuit = models.transmon(basis,Ej,Ec,sparse=False)
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson
kerman = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [ ]:
circuit = models.transmon([100],Ej,Ec,sparse=False)
H_LC=circuit.chargeHamiltonianLC()
H_J=circuit.josephsonCharge
charge = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [ ]:
kerman = kerman.detach().numpy()
charge = charge.detach().numpy()
plotCompare(arange(len(kerman)),{'kerman':kerman,'charge':charge},'Energy Spectrum','levels','E(GHz)')